In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import requests
import time

In [2]:
orders = pd.read_csv("orders.csv", low_memory=False)
archive_orders = pd.read_csv("archive_orders.csv", low_memory=False)
bundle_orders = pd.read_csv("bundle_orders.csv", low_memory=False)

In [3]:
final_orders = pd.concat([archive_orders, orders, bundle_orders], ignore_index=True)
final_orders = final_orders[final_orders['delivery_date'] != '0001-11-30']
final_orders['delivery_date'] = final_orders['delivery_date'].str.replace('0022', '2022', regex=False)
final_orders['delivery_date'] = pd.to_datetime(final_orders['delivery_date'])

In [4]:
final_orders['day'] = final_orders['delivery_date'].dt.day
final_orders['month'] = final_orders['delivery_date'].dt.month
final_orders['year'] = final_orders['delivery_date'].dt.year

In [5]:
dates = final_orders.groupby(['year', 'month', 'day'])['delivery_date'].count().reset_index()
dates = dates[['year', 'month', 'day']]

In [6]:
all_holidays = []
for index, row in dates.iterrows():
    url = f"{base_url}&year={row.year}&month={row.month}&day={row.day}"
    response = requests.get(url)
    if response.status_code == 200:
        j = response.json()
        holidays = pd.DataFrame(j)
        all_holidays.append(holidays)
    else:
        print(f"Failed to fetch holidays for year {year}. Status code: {response.status_code}")
    time.sleep(2)
holiday_list = pd.concat(all_holidays, ignore_index=True)

In [7]:
holiday_list.to_csv('holiday-list.csv', index=True)